In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *


Using TensorFlow backend.


In [2]:
df_test = pd.read_csv('../dogdata/sample_submission.csv')


In [3]:
import os
dirname='../dogdata/new_train/'
folders=os.listdir(dirname)
data=[]
label=[]

# os.path.join(dirname,'0')
for folder in folders:
    label+=[str(folder)]
label= sorted(label, key=str.lower)
i=[i for i in range(120)]
label_map = dict(zip(label, i))
label_map

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8,
 'beagle': 9,
 'bedlington_terrier': 10,
 'bernese_mountain_dog': 11,
 'black-and-tan_coonhound': 12,
 'blenheim_spaniel': 13,
 'bloodhound': 14,
 'bluetick': 15,
 'border_collie': 16,
 'border_terrier': 17,
 'borzoi': 18,
 'boston_bull': 19,
 'bouvier_des_flandres': 20,
 'boxer': 21,
 'brabancon_griffon': 22,
 'briard': 23,
 'brittany_spaniel': 24,
 'bull_mastiff': 25,
 'cairn': 26,
 'cardigan': 27,
 'chesapeake_bay_retriever': 28,
 'chihuahua': 29,
 'chow': 30,
 'clumber': 31,
 'cocker_spaniel': 32,
 'collie': 33,
 'curly-coated_retriever': 34,
 'dandie_dinmont': 35,
 'dhole': 36,
 'dingo': 37,
 'doberman': 38,
 'english_foxhound': 39,
 'english_setter': 40,
 'english_springer': 41,
 'entlebucher': 42,
 'eskimo_dog': 43,
 'flat-coated_retriever': 44,
 'french_bulldog': 45,
 'german_shepherd'

In [4]:
targets_series = pd.Series(label)
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
dim = 256

In [6]:
x_train = []

y_train = []
# Preparing training data
dirs = os.listdir("../dogdata/new_train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("../dogdata/new_train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('../dogdata/new_train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(120)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train,np.float32)

print(x_train.shape)
print(y_train.shape)


100%|██████████| 120/120 [00:25<00:00,  4.94it/s]


(10222, 256, 256, 3)
(10222, 120)


In [7]:
#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = x_train[train_index], x_train[test_index]
    y_train, y_valid = y_train[train_index], y_train[test_index]



Using 8586 for training and 1636 for validation


In [8]:
POOLING=None
print((x_train.shape, x_valid.shape, y_train.shape, y_valid.shape))
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
train_x_bf = xception_bottleneck.predict(x_train, batch_size=32, verbose=1)
valid_x_bf = xception_bottleneck.predict(x_valid, batch_size=32, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

((8586, 256, 256, 3), (1636, 256, 256, 3), (8586, 120), (1636, 120))
1636/1636 [==============================] - 32s    
Xception train bottleneck features shape: (8586, 8, 8, 2048) size: 1,125,384,192
Xception valid bottleneck features shape: (1636, 8, 8, 2048) size: 214,433,792


In [14]:
model = Sequential()
model.add(Flatten(input_shape=train_x_bf.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Dense(120, activation='softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x_bf, y_train,
              epochs=30,
              batch_size=10,
              validation_data=(valid_x_bf, y_valid))

Train on 8586 samples, validate on 1636 samples
Epoch 1/30
8586/8586 [==============================] - 43s - loss: 5.5432 - acc: 0.0156 - val_loss: 4.6445 - val_acc: 0.0391- loss: 5.6041 - acc: 0 - ETA: 4s - loss: 5.59 - ETA: 3s - loss: 5.5 - ETA: 2s - loss: - ETA
Epoch 2/30
8586/8586 [==============================] - 43s - loss: 4.8424 - acc: 0.0269 - val_loss: 4.4966 - val_acc: 0.0672
Epoch 3/30
8586/8586 [==============================] - 42s - loss: 4.6319 - acc: 0.0337 - val_loss: 4.4828 - val_acc: 0.0611
Epoch 4/30
8586/8586 [==============================] - 42s - loss: 4.5122 - acc: 0.0446 - val_loss: 4.4089 - val_acc: 0.0758
Epoch 5/30
8586/8586 [==============================] - 42s - loss: 4.4369 - acc: 0.0518 - val_loss: 4.3872 - val_acc: 0.0697
Epoch 6/30
8586/8586 [==============================] - 42s - loss: 4.3980 - acc: 0.0608 - val_loss: 4.3307 - val_acc: 0.0990
Epoch 7/30
8586/8586 [==============================] - 42s - loss: 4.3666 - acc: 0.0623 - val_loss: 4.2

In [15]:
x_test = []
for f in tqdm(df_test['id'].values, miniters=100):
    img = cv2.imread('../dogdata/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
test_x_bf = xception_bottleneck.predict(x_test, batch_size=32, verbose=1)

100%|██████████| 10357/10357 [00:24<00:00, 430.76it/s]


(10357, 256, 256, 3)
10357/10357 [==============================] - 207s   


In [16]:
preds = model.predict(test_x_bf,verbose=1)
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.to_csv('dogbottlenecknsubmission.csv', index=False)

10336/10357 [============================>.] - ETA: 0s

<a id="s"></a>
### Second Approach

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *


Using TensorFlow backend.


In [2]:


df_test = pd.read_csv('../dogdata/sample_submission.csv')


In [3]:
#label=pd.read_csv('C:/Users/Tamim/Documents/Finetune/labels.csv')

In [4]:
import os
dirname='../dogdata/new_train/'
folders=os.listdir(dirname)
data=[]
label=[]

# os.path.join(dirname,'0')
for folder in folders:
    label+=[str(folder)]
label= sorted(label, key=str.lower)
i=[i for i in range(120)]
label_map = dict(zip(label, i))
label_map

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8,
 'beagle': 9,
 'bedlington_terrier': 10,
 'bernese_mountain_dog': 11,
 'black-and-tan_coonhound': 12,
 'blenheim_spaniel': 13,
 'bloodhound': 14,
 'bluetick': 15,
 'border_collie': 16,
 'border_terrier': 17,
 'borzoi': 18,
 'boston_bull': 19,
 'bouvier_des_flandres': 20,
 'boxer': 21,
 'brabancon_griffon': 22,
 'briard': 23,
 'brittany_spaniel': 24,
 'bull_mastiff': 25,
 'cairn': 26,
 'cardigan': 27,
 'chesapeake_bay_retriever': 28,
 'chihuahua': 29,
 'chow': 30,
 'clumber': 31,
 'cocker_spaniel': 32,
 'collie': 33,
 'curly-coated_retriever': 34,
 'dandie_dinmont': 35,
 'dhole': 36,
 'dingo': 37,
 'doberman': 38,
 'english_foxhound': 39,
 'english_setter': 40,
 'english_springer': 41,
 'entlebucher': 42,
 'eskimo_dog': 43,
 'flat-coated_retriever': 44,
 'french_bulldog': 45,
 'german_shepherd'

In [5]:
targets_series = pd.Series(label)
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dim = 256

In [7]:
x_train = []

y_train = []
# Preparing training data
dirs = os.listdir("../dogdata/new_train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("../dogdata/new_train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('../dogdata/new_train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(120)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train,np.float32)

print(x_train.shape)
print(y_train.shape)


100%|██████████| 120/120 [00:23<00:00,  5.01it/s]


(10222, 256, 256, 3)
(10222, 120)


In [8]:
#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = x_train[train_index], x_train[test_index]
    y_train, y_valid = y_train[train_index], y_train[test_index]



Using 8586 for training and 1636 for validation


In [9]:
datagen = ImageDataGenerator( featurewise_center=False,  # set input mean to 0 over the dataset
                            samplewise_center=False,  # set each sample mean to 0
                            featurewise_std_normalization=False,  # divide inputs by std of the dataset
                            samplewise_std_normalization=False,  # divide each input by its std
                            zca_whitening=False,  # apply ZCA whitening
                            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                            zoom_range = 0.1, # Randomly zoom image 
                            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                            horizontal_flip=True, 
                            vertical_flip=False)
                                      


In [15]:
weights = os.path.join('', 'weights.h5')

epochs = 5
learning_rate = 0.0001
batch_size = 32

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]



In [11]:
base_model = Xception(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
predictions = Dense(120, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#model = Sequential()
#model.add(Conv2D(filters = 2, kernel_size = (5,5),padding = 'Same', 
#                 activation ='relu', input_shape = (dim,dim,3)))
#model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Flatten())
#model.add(Dense(256, activation = "relu"))
#model.add(Dropout(0.5))
#model.add(Dense(120, activation = "softmax"))



82665472/83683744 [============================>.] - ETA: 0s

In [16]:
# Freeze layers not in classifier due to loading imagenet weights
for layer in base_model.layers:
    layer.trainable = False

# print(model.summary())

# Load any existing weights
# if os.path.isfile(weights):
#     model.load_weights(weights)
    


In [17]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])


In [18]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=epochs, 
                    verbose=1)

Epoch 1/5
269/268 [==============================] - 187s - loss: 0.4990 - acc: 0.8708 - val_loss: 0.4639 - val_acc: 0.8661
Epoch 2/5
269/268 [==============================] - 184s - loss: 0.4314 - acc: 0.8890 - val_loss: 0.4465 - val_acc: 0.8667
Epoch 3/5
269/268 [==============================] - 184s - loss: 0.3909 - acc: 0.8943 - val_loss: 0.4387 - val_acc: 0.8698
Epoch 4/5
269/268 [==============================] - 183s - loss: 0.3502 - acc: 0.9037 - val_loss: 0.4518 - val_acc: 0.8704
Epoch 5/5
269/268 [==============================] - 183s - loss: 0.3485 - acc: 0.9039 - val_loss: 0.3945 - val_acc: 0.8912


In [19]:
x_test = []
for f in tqdm(df_test['id'].values, miniters=100):
    img = cv2.imread('../dogdata/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)


100%|██████████| 10357/10357 [00:22<00:00, 455.03it/s]


(10357, 256, 256, 3)


In [20]:
preds = model.predict(x_test,verbose=1)
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.to_csv('dogfinetunecorfr5submission.csv', index=False)

10357/10357 [==============================] - 200s   
